# Deep Learning with PyTorch

Modified from the version created by Dr. [Jian Tao](https://orcid.org/0000-0003-4228-6089), Texas A&M University

Oct 3, 2023

## Deep Learning
Deep Learning (DL) is one cagegory of machine learning methods that are based on artificial neural networks to improve computer algorithms automatically through data.

DL methods can be devided into:

* Supervised Learning
    * trained with labeled data; including regression and classification problems
* Unsupervised Learning
    * trained with unlabeled data; clustering and association rule learning problems.
* Reinforcement Learning
    * no training data; stochastic Markov decision process; robotics and self-driving cars.

DL methods are useful primarily for the following reasons:

* DL is computationally expensive, but it is capable of handling high dimensional data.
* feature extraction is done automatically.

![Deep Learning](https://github.com/happidence1/AILabs/blob/master/images/deeplearning.svg?raw=1)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time  # Import the time module

In [2]:
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
net = Net().to(device)
# net = Net()

In [5]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
# Download and prepare the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [7]:
# Timing start
start_time = time.time()
# Training loop
for epoch in range(10):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}")
            running_loss = 0.0

print("Finished Training")
# Timing end
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time} seconds")

[1, 200] loss: 2.083
[1, 400] loss: 0.861
[1, 600] loss: 0.436
[1, 800] loss: 0.352
[2, 200] loss: 0.249
[2, 400] loss: 0.235
[2, 600] loss: 0.201
[2, 800] loss: 0.188
[3, 200] loss: 0.153
[3, 400] loss: 0.147
[3, 600] loss: 0.117
[3, 800] loss: 0.120
[4, 200] loss: 0.104
[4, 400] loss: 0.097
[4, 600] loss: 0.095
[4, 800] loss: 0.089
[5, 200] loss: 0.087
[5, 400] loss: 0.083
[5, 600] loss: 0.078
[5, 800] loss: 0.077
[6, 200] loss: 0.072
[6, 400] loss: 0.072
[6, 600] loss: 0.068
[6, 800] loss: 0.063
[7, 200] loss: 0.061
[7, 400] loss: 0.062
[7, 600] loss: 0.063
[7, 800] loss: 0.062
[8, 200] loss: 0.051
[8, 400] loss: 0.056
[8, 600] loss: 0.056
[8, 800] loss: 0.059
[9, 200] loss: 0.051
[9, 400] loss: 0.047
[9, 600] loss: 0.053
[9, 800] loss: 0.051
[10, 200] loss: 0.042
[10, 400] loss: 0.049
[10, 600] loss: 0.046
[10, 800] loss: 0.047
Finished Training
Training took 49.51250243186951 seconds


In [8]:
# Save the trained model
torch.save(net.state_dict(), 'mnist_net.pth')

In [9]:
# Load the trained model for evaluation
net = Net()
net.load_state_dict(torch.load('mnist_net.pth'))
net.eval()  # Set the model to evaluation mode

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [10]:
# Download and prepare the test dataset
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [11]:
correct = 0
total = 0

# Disable gradient computation during evaluation
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total:.2f}%')

Accuracy on the test set: 98.54%
